In [1]:
! pip install huggingface_hub
! git config --global credential.helper store

In [27]:
from huggingface_hub import notebook_login
from huggingface_hub import HfFolder
import pandas as pd



In [6]:
#enter your API key, you can make one for free on HF
notebook_login()

Login successful
Your token has been saved to /Users/siennaaa/.huggingface/token


In [28]:
from huggingface_hub import InferenceApi

inference = InferenceApi("bigscience/bloom",token=HfFolder.get_token())

In [29]:
exp_condition_df = pd.read_csv('../experiments/analysis_of_prior_work/tasks_collection_essentialism.csv')
exp_condition_df = exp_condition_df.fillna("")
exp_condition_df


,paper,study,type,type_category,stat_test_original,stat_test_new,is_natural_kind,condition,mask_pred_1,mask_pred_2,...,question_1_5_text,question_1_5_mask,question_1_6_type,question_1_6_text,question_1_6_mask,scenario_2,question_2_1_type,question_2_1_text,question_2_1_mask,hypotheses
0,rose_nichols_2019,1,transformation,superficial,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_preserved,bee,spider,...,,,,,,,,,,There will be a difference between purpose cha...
1,rose_nichols_2019,1,transformation,superficial,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_changed,bee,spider,...,,,,,,,,,,There will be a difference between purpose cha...
2,rose_nichols_2019,2,transformation,inside_removal,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_preserved,bee,spider,...,,,,,,,,,,There will be a difference between purpose cha...
3,rose_nichols_2019,2,transformation,inside_removal,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_changed,bee,spider,...,,,,,,,,,,There will be a difference between purpose cha...
4,rose_nichols_2019,3,nature_nurture,environment,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_preserved,bee,spider,...,,,,,,,,,,There will be a difference between purpose cha...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
126,barton_and_komatsu_1997,1,transformation,functional,"2*(word type: natural kinds, artifacts) x 2*(c...",glm(response ~ word type*change type),False,tire_changed_in_functional,tire,,...,,,,,,,,,,Molecular changes will have a greater impact o...
127,barton_and_komatsu_1997,1,transformation,physical,"2*(word type: natural kinds, artifacts) x 2*(c...",glm(response ~ word type*change type),False,tire_changed_in_physical,tire,,...,,,,,,,,,,Molecular changes will have a greater impact o...
128,barton_and_komatsu_1997,1,transformation,molecular,"2*(word type: natural kinds, artifacts) x 2*(c...",glm(response ~ word type*change type),False,mirror_changed_in_molecule,mirror,,...,,,,,,,,,,Molecular changes will have a greater impact o...
129,barton_and_komatsu_1997,1,transformation,functional,"2*(word type: natural kinds, artifacts) x 2*(c...",glm(response ~ word type*change type),False,mirror_changed_in_functional,mirror,,...,,,,,,,,,,Molecular changes will have a greater impact o...


In [57]:
def bloom_infer(prompt, seed=2022):
    # top_k = None if top_k == 0 else top_k
    # do_sample = False if num_beams > 0 else not greedy_decoding
    # num_beams = None if (greedy_decoding or num_beams == 0) else num_beams
    # no_repeat_ngram_size = None if num_beams is None else no_repeat_ngram_size
    # top_p = None if num_beams else top_p
    # early_stopping = None if num_beams is None else num_beams > 0

    params = {
        "max_new_tokens": 5,
        "top_k": None,
        "top_p": 0.9,
        "temperature": 0.7,
        "do_sample": True,
        "seed": seed,
        "early_stopping": False,
        "no_repeat_ngram_size": None,
        "num_beams": None,
        "eos_token_id": None,
        "length_penalty": 1.0,
    }
    

    response = inference(prompt, params=params)

    return response

In [58]:

bloom_response_df = pd.DataFrame(columns=exp_condition_df.columns)

for exp_condition in range(...,...): #range(2): #range[1] means the second row of the csv file, can change to range[2] to get the third row of the csv file. We will want to start with [1] since we ran the first row already.
    for rep_id in range(50): #need to change to 50 to generate 50 responses
      exp_row = exp_condition_df.loc[[exp_condition]]
      exp_row["rep_id"] = rep_id + 1

      prompt = ""
      for scenario_id in range(1, 3):  # We have two scenarios.
          scenario_column_name = f"scenario_{scenario_id}"  # f"scenario_{scenario_id}" is scenario column name
          # print(exp_condition_df[scenario_column_name][exp_condition])
          if exp_condition_df[scenario_column_name][exp_condition] == "":
            break  # Skip when the scenario is empty.
          # prompt += "Read the following context and answer the question in one sentence.\n\n"
          prompt += exp_condition_df[scenario_column_name][exp_condition].strip()

          for question_id in [1]:            #range(1,7): #some cases have more than one question, so we need to loop through them. If there are none, we will break.
            if scenario_id == 1:
              continue
            question_column_name = f"question_{scenario_id}_{question_id}_text"  # f"question_{scenario_id}_{question_id}_text" is question column name
            if question_column_name not in exp_condition_df:  
              break
            if exp_condition_df[question_column_name][exp_condition] == "":
              break

            # prompt += "\n\nQuestion: " + exp_condition_df[question_column_name][exp_condition].strip() + "\nAnswer:"

            if scenario_id == 1:
              prompt += " " + exp_condition_df[question_column_name][exp_condition].strip() + " I think it will be "
            
            if scenario_id == 2:
              # prompt += " When they were finished, was this animal a zebra or a horse? I think it was a"
              # prompt += " Is the thing after the special operation a spider or bee? I think it is a"
              prompt += " " + exp_condition_df[question_column_name][exp_condition].strip() + " I think the things will be "
              # prompt += " " + exp_condition_df[f"question_{scenario_id}_{question_id}_mask"][exp_condition].strip() + "I think it was a"
              # prompt += " When that seed grew, the thing popped out of the ground was"
            else:
              # prompt += "\n\nQuestion: " + exp_condition_df[question_column_name][exp_condition].strip() + "\nAnswer:"
              prompt += " " + exp_condition_df[question_column_name][exp_condition].strip()

            
            # print(f"Scenario: {scenario_id}, Question: {question_id}:\n{prompt}\n\n")
            # print("-------------------------------------------------------------------\n")
            exp_row[f"bloom_prompt_question{scenario_id}_{question_id}"] = prompt
            print(prompt)
            response = bloom_infer(prompt, seed=rep_id)
            answer = response[0]["generated_text"][len(prompt):].strip()
            print(answer)

            bloom_response_column_name = f"bloom_response_question_{scenario_id}_{question_id}" 
            exp_row[bloom_response_column_name] = answer

            prompt += answer + "\n"

      bloom_response_df = pd.concat([bloom_response_df, exp_row], axis=0)

bloom_response_df.rep_id = bloom_response_df.rep_id.astype(int)

col = bloom_response_df.pop("rep_id")
bloom_response_df.insert(2, col.name, col)

bloom_response_df



Some very talented and skilled scientists, Suzy and Andy, decide that they are going to perform a special operation on a queen bee. They removed its wings and antennae, lengthened its legs and added a new pair of legs. They also inserted into the back of it something for making webs and trained the animal so that it would eat insects. After the special operation, this thing looked like a spider. After running some tests, they found that the thing after the special operation didn’t spin webs to catch insects and eat them. Instead, it only pollinated flowers and made honey.The scientists now want to know what the thing's offspring will be like. So they devise a special technique to have its eggs fertilized by a bee. Will the things that hatch from the eggs be bees or spiders? I think the things will be 
spiders.
Some very talented and skilled scientists, Suzy and Andy, decide that they are going to perform a special operation on a queen bee. They removed its wings and antennae, lengthene

,paper,study,rep_id,type,type_category,stat_test_original,stat_test_new,is_natural_kind,condition,mask_pred_1,...,question_1_6_type,question_1_6_text,question_1_6_mask,scenario_2,question_2_1_type,question_2_1_text,question_2_1_mask,hypotheses,bloom_prompt_question2_1,bloom_response_question_2_1
6,rose_nichols_2019,4,1,nature_nurture,DNA,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_preserved_fertilized_by_bee,bee,...,,,,The scientists now want to know what the thing...,inheritance,Will the things that hatch from the eggs be be...,,There will be a lower spider ratings when the ...,"Some very talented and skilled scientists, Suz...",spiders.
6,rose_nichols_2019,4,2,nature_nurture,DNA,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_preserved_fertilized_by_bee,bee,...,,,,The scientists now want to know what the thing...,inheritance,Will the things that hatch from the eggs be be...,,There will be a lower spider ratings when the ...,"Some very talented and skilled scientists, Suz...",spiders. But I
6,rose_nichols_2019,4,3,nature_nurture,DNA,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_preserved_fertilized_by_bee,bee,...,,,,The scientists now want to know what the thing...,inheritance,Will the things that hatch from the eggs be be...,,There will be a lower spider ratings when the ...,"Some very talented and skilled scientists, Suz...",
6,rose_nichols_2019,4,4,nature_nurture,DNA,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_preserved_fertilized_by_bee,bee,...,,,,The scientists now want to know what the thing...,inheritance,Will the things that hatch from the eggs be be...,,There will be a lower spider ratings when the ...,"Some very talented and skilled scientists, Suz...",
6,rose_nichols_2019,4,5,nature_nurture,DNA,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_preserved_fertilized_by_bee,bee,...,,,,The scientists now want to know what the thing...,inheritance,Will the things that hatch from the eggs be be...,,There will be a lower spider ratings when the ...,"Some very talented and skilled scientists, Suz...",spiders.Because bees
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,rose_nichols_2019,4,46,nature_nurture,DNA,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_changed_fertilized_by_spider,bee,...,,,,The scientists now want to know what the thing...,inheritance,Will the things that hatch from the eggs be be...,,There will be a higher spider ratings when the...,"Some very talented and skilled scientists, Suz...",spiders. The thing
9,rose_nichols_2019,4,47,nature_nurture,DNA,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_changed_fertilized_by_spider,bee,...,,,,The scientists now want to know what the thing...,inheritance,Will the things that hatch from the eggs be be...,,There will be a higher spider ratings when the...,"Some very talented and skilled scientists, Suz...",
9,rose_nichols_2019,4,48,nature_nurture,DNA,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_changed_fertilized_by_spider,bee,...,,,,The scientists now want to know what the thing...,inheritance,Will the things that hatch from the eggs be be...,,There will be a higher spider ratings when the...,"Some very talented and skilled scientists, Suz...",spiders because the thing
9,rose_nichols_2019,4,49,nature_nurture,DNA,"t.test(response, telos)",chisq.test(response ~ telos),True,bee_telo_changed_fertilized_by_spider,bee,...,,,,The scientists now want to know what the thing...,inheritance,Will the things that hatch from the eggs be be...,,There will be a higher spider ratings when the...,"Some very talented and skilled scientists, Suz...",spiders.Why?


In [51]:
# print("\n==========================\n".join(bloom_response_df["bloom_response_question_1_1"].tolist()))


KeyError: 'bloom_response_question_1_1'

In [59]:
bloom_response_df.to_csv("../../data/analysis_of_prior_work/.../....csv")